In [ ]:
import numpy as np
import importlib, os, datetime
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation


from quick_sim import setup_sim




# import edward_tools.fq_runner as fq_runner
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, coupled_flux_qubit_non_linear_approx_pot_break_down
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from NAND_PARAMETERS import *
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation


coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

# parameters

### circuit parameters

In [ ]:
has_velocity = True
PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 7
k_BT = k_B * T
time_scale = 1.0
C_factor, L_factor, I_m_factor = 1, 1, 0
U0_1, U0_2 = 100, 100

gamma = 10

I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp                                     # ohm
L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 
R_1, R_2 = 1, 1 # ohm
C_1, C_2 = 500e-15, 500e-15                             # F

m_c, m_1, m_2 = C_1, C_1, C_2
x_c = PHI_0 / (2 * np.pi)
t_c = np.sqrt(L_1 * C_1)

v_1, v_2, v_3, v_4 = np.zeros(4)
beta_1, beta_2, d_beta_1, d_beta_2, = np.zeros(4)
_lambda, _theta, _eta = np.zeros(4), np.zeros(4), np.zeros(4)


In [ ]:
m_c

In [ ]:
def parameterGenerator(_T, L = None, C = None, R = None, external_parameter = None):
    global k_BT, U0_1, U0_2, gamma, T, C_1, C_2, L_1, L_2, m_c, m_1, m_2
    global R_1, R_2, m_c, m_1, m_2, t_c, beta_1, beta_2, d_beta_1, d_beta_2, _lambda, _theta, _eta
    global v_1, v_2, v_3, v_4
    T = _T
    k_BT = k_B * T
    if R: R_1, R_2 = R, R # ohm
    
    if C: C_1, C_2 = C, C

    if L: L_1, L_2 = L, L

    m_c, m_1, m_2 = C_1, C_1, C_2
    
    t_c = np.sqrt(L_1 * C_1)
    v_c = x_c / t_c
    
    U0_1 = m_c * x_c**2 / t_c**2 / k_BT
    U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
    kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

    lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
    theta_1  = 1
    eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

    lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
    theta_2  = 1 / (C_2/C_1)
    eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

    lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
    theta_3  = 4
    eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

    lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
    theta_4  = 4 / (C_2/C_1)
    eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

    gamma = 10

    # beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
    # beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;
    beta_1, beta_2 = 2.3, 2.3
    

    d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
    d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;

    _lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4])
    _theta  = np.array([theta_1, theta_2, theta_3, theta_4])
    _eta  =   np.array([eta_1, eta_2, eta_3, eta_4])
    
    v_1 = np.random.normal(0, np.sqrt(k_BT/m_1)) / v_c
    v_2 = np.random.normal(0, np.sqrt(k_BT/m_2)) / v_c
    v_3 = np.random.normal(0, np.sqrt(k_BT/(m_1/4))) / v_c
    v_4 = np.random.normal(0, np.sqrt(k_BT/(m_2/4))) / v_c
    
    return {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": external_parameter['phi_1_x'],  "phi_2_x": external_parameter['phi_2_x'],  
        "phi_1_dcx": external_parameter['phi_1_dcx'],  "phi_2_dcx": external_parameter['phi_2_dcx'],
        "M_12": external_parameter['M_12'], 'x_c': x_c
    }


In [ ]:
U0_1, C_1, m_c

In [ ]:
external_parameter = {
    "phi_1_x": 0,  "phi_2_x": 0,  "phi_1_dcx": 0,  "phi_2_dcx": 0, "M_12": 0
}
initial_parameter_dict = parameterGenerator(10, L = 140e-12, C = 500e-15, R = 1, 
                                            external_parameter=external_parameter)

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1_00
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1



In [ ]:
"""
# step 1: Define potential
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]

coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])
contour_range = [300, 2000]
    
coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)



### protocol setting

In [ ]:
zeroDissipation = False
# zeroDissipation = True

saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]
params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT

if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "phi_1_x_on": phi_1_x_on_12, "phi_2_x_on": phi_1_x_on_12,
    "phi_1_dcx_on": phi_1_dcx_on_12, "phi_2_dcx_on": phi_2_dcx_on_12, "M_12_on": M_12_on,
    "gamma": gamma
}


# bookmark
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
phi_1_dcx_off, phi_2_dcx_off, M_12_off

m_12_factor = 1
CE_1_a = {
    "phi_1_x": phi_1_x_on_12 * 0.8, "phi_2_x": phi_2_x_on_12, "M_12": M_12_on, \
    "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on_12, "name":"CE_1"
}
CE_1_b = {
    "phi_1_x": phi_1_x_on_12  , "phi_2_x": phi_2_x_on_12, "M_12": M_12_on * m_12_factor, \
    "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on_12, "name":"CE_1"
}


# longer CE
t_duration = 100
protocol_list_LCE = [
    create_CE_Protocol(t_duration, CE_8),
    create_CE_Protocol(t_duration, CE_1),
    create_CE_Protocol(t_duration, CE_6),
    create_CE_Protocol(t_duration, CE_3),
    create_CE_Protocol(t_duration, CE_8),
    create_CE_Protocol(t_duration, four_well)
]
# shorter CE
protocol_list = [
    # forward
    create_CE_Protocol(50, CE_1_a),
    create_CE_Protocol(150, CE_1_b),
    create_CE_Protocol(150, CE_1_a),
    create_CE_Protocol(50, four_well)
    # create_CE_Protocol(100, CE_8),
    # create_CE_Protocol(100, flip_V),
    # create_CE_Protocol(100, flip_H),

    # create_CE_Protocol(100, CE_3),
    # create_CE_Protocol(100, CE_8)
]


protocol_list_variable_duration = [
    # forward
    create_CE_Protocol(30, CE_1_a),
    create_CE_Protocol(60, CE_1_b),
    create_CE_Protocol(30, CE_1_a),
    create_CE_Protocol(100, four_well)

]


protocol_list_fixed_duration = [
    # forward
    create_CE_Protocol(100, CE_1_b),
    # create_CE_Protocol(100, CE_1_b),
    # create_CE_Protocol(100, CE_1_a),
    create_CE_Protocol(100, four_well)

]



protocol_list_shape_check = [
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_off  , "phi_2_x": phi_2_x_off, "M_12": M_12_on, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name":"M_12_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_off  , "phi_2_x": phi_2_x_off, "M_12": -M_12_on, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name":"M_12_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_on_12  , "phi_2_x": phi_2_x_off, "M_12": -M_12_off, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name": "phi_1x_on_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": -phi_1_x_on_12  , "phi_2_x": phi_2_x_off, "M_12": -M_12_off, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name": "phi_1x_on_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_off  , "phi_2_x": phi_2_x_on_12, "M_12": -M_12_off, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name": "phi_1x_on_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_off  , "phi_2_x": -phi_2_x_on_12, "M_12": -M_12_off, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name": "phi_1x_on_only"
    })
]

protocol_list = protocol_list_shape_check

In [ ]:

t_factor = 1
protocol_list_variable_duration = [
    # forward
    # create_CE_Protocol(50  * t_factor, CE_1, ratio_array),
    create_CE_Protocol(50 * t_factor, CE_1),
    # create_CE_Protocol(20 * t_factor, CE_1_a),
    create_CE_Protocol(50 * t_factor, four_well),
    # create_CE_Protocol(95 * t_factor, CE_1_b),
    # create_CE_Protocol(95 * t_factor, CE_1_a),
    # create_CE_Protocol(5  * t_factor, four_well)

]

protocol_list = protocol_list_variable_duration

In [ ]:
def plot_3_well_potential(phi_1x, mu):
    phi_1_array = np.linspace(-4, 4, 1000)
    phi_2 = 2

    params_list_used = cfqr.protocol.get_params(0)
    mu_array = np.linspace(0, 0.15, 10)
    phi_1x_array  = np.linspace(0., 0.3, 10)
    for x in phi_1xdc_array:
        fig = plt.figure(figsize= [20, 5])
        ax_0 = fig.add_subplot(1, 4, 1)
        ax_1 = fig.add_subplot(1, 4, 2)
        ax_2 = fig.add_subplot(1, 4, 3)
        ax_3 = fig.add_subplot(1, 4, 4)

        params_list_used[8] = 0.61 # phi_1x
        params_list_used[9] = 0.10 # phi_2x"
        params_list_used[10] = 0 # phi_1xdc
        params_list_used[11] = 1.79 # phi_2xdc
        params_list_used[12] = 0.06 # mu
        
        params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(cfqr.protocol_key[-6:-1], params_list_used[-6:-1])]
        fig.suptitle(f"phi_1xdc = {params_list_used[10]:.3g}, phi_1x = {params_list_used[8]:.3g}, phi_2x = {params_list_used[9]:.3g}, mu = {params_list_used[12]:.3g}") 

        potential_min = []
        for _phi_1dc in phi_1dc_array[0:]:
            potential = cfqr.potential.potential(phi_1_array, phi_2, _phi_1dc, phi_2dc, params_list_used)
            ax_0.plot(phi_1_array, np.full(len(phi_1_array), fill_value=_phi_1dc), potential, label = f"{_phi_1dc:.3g}")
            ax_1.plot(phi_1_array, potential, label = f"phi_1dc = {_phi_1dc:.3g}")
            
            potential = cfqr.potential.potential(phi_1_array, -phi_2, _phi_1dc, phi_2dc, params_list_used)
            ax_2.plot(phi_1_array, np.full(len(phi_1_array), fill_value=_phi_1dc), potential, label = f"{_phi_1dc:.3g}")
            ax_3.plot(phi_1_array, potential, label = f"phi_1dc = {_phi_1dc:.3g}")
            
            
        ax_0.set_xlim(-4, 4)
        # ax_0.set_zlim(min(potential_min) - 100, min(potential_min)+1000)
        ax_1.set_ylim( -1700, -1000)
        ax_3.set_ylim( -1700, -1000)
        # ax_1.legend()
        plt.show()

# first initialization

### create initial state 

In [ ]:
%%capture
initial_potential_protocol_list = [create_CE_Protocol(5, four_well)]
init_state_saved = cfq_batch_sweep.create_initial_state(initial_parameter_dict, initial_potential_protocol_list, coupled_fq_pot, params)

In [ ]:

t_factor = 1
protocol_list_variable_duration = [
    # forward
    # create_CE_Protocol(50  * t_factor, CE_1, ratio_array),
    create_CE_Protocol(50 * t_factor, {
        "phi_1_x": phi_1_x_on_12, "phi_2_x": phi_2_x_on_12, "M_12": M_12_on, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on_12 * 1.01, "name":"CE_1"
    }),
    # create_CE_Protocol(20 * t_factor, CE_1_a),
    create_CE_Protocol(50 * t_factor, four_well),
    # create_CE_Protocol(95 * t_factor, CE_1_b),
    # create_CE_Protocol(95 * t_factor, CE_1_a),
    # create_CE_Protocol(5  * t_factor, four_well)

]

protocol_list = protocol_list_variable_duration

### create cfqr object

In [ ]:
"""
# step 3: create the relevant storage protocol and computation protocol
"""
def modifiedFunction(t, result):
    return result


computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity)

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
cfqr.set_sim_attributes(init_state=init_state_saved)

In [ ]:
# importlib.reload(coupled_fq_protocol_library)

# L = 140e-12
# C = 500e-15
# T = 7
# external_parameter_list = {
#     "bifurcation": { "phi_1_x": 0.59,  "phi_2_x": 0.1,  "phi_1_dcx": 0.0,  "phi_2_dcx": 1.79, "M_12": 0.06 },
#     "four_well":   { "phi_1_x": 0,  "phi_2_x": 0,  "phi_1_dcx": 0.0,  "phi_2_dcx": 0, "M_12": 0.00 }
# }

# parameter_choice = 'bifurcation'
# # parameter_choice = "four_well"

# external_parameter = external_parameter_list[parameter_choice]
# initial_parameter_dict = parameterGenerator(_T = T, L = L, C = C, R = 1, 
#                                             external_parameter=external_parameter)
# cutlineDirection = "v"
# cutlineValue = 2
# cutline_data = coupled_fq_protocol_library.get_potential_along_a_1D_cutline_by_overriding_parameters(cfqr, parameter_data=initial_parameter_dict, 
#                                                                                                      cutlineDirection=cutlineDirection, cutlineValue=cutlineValue)

In [ ]:
# adjusted_targetU = cutline_data["targetU"] - np.min(cutline_data["targetU"] )
# # L = [5e-12, 10e-12, 15e-12, 20e-12, 25e-12]
# L = [140e-12]
# for _L in L:
#     initial_parameter_dict = parameterGenerator(_T = T, L = _L, C = C, R = 1, 
#                                             external_parameter=external_parameter)
#     cutline_data = coupled_fq_protocol_library.get_potential_along_a_1D_cutline_by_overriding_parameters(cfqr, parameter_data=initial_parameter_dict, 
#                                                                                                      cutlineDirection=cutlineDirection, cutlineValue=cutlineValue)
#     adjusted_targetU = cutline_data["targetU"] - np.min(cutline_data["targetU"] )
#     x = plt.plot(cutline_data["plotAxis"], adjusted_targetU, label = f"L = {_L/1e-12:.3g}pH, bh = {max(adjusted_targetU[40:60]):.3g}$k_BT$")

    
# _c_direction = {"v": r"$\varphi_1$", "h": r"$\varphi_2$"}
# _perp_direction = "h" if cutlineDirection == "v" else cutlineDirection


# plt.title(f"U along {_c_direction[cutlineDirection]} = {cutlineValue} (T = {T}K, {parameter_choice})")
# plt.xlim(-4, 4)
# plt.ylim(-1, 20)
# plt.ylabel(r"Potential ($k_BT$)", fontsize = 12)
# plt.xlabel(_c_direction[_perp_direction], fontsize = 12)
# plt.legend()
# plt.show()


# check before simulation

### critical points monitor

In [ ]:
critical_point_data = cfqr.criticalPointMonitor(np.s_[::100])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = [10, 3])
ax[0].plot(critical_point_data["t_array"], critical_point_data["00_01"]["PE"], label = "00_01")
ax[0].plot(critical_point_data["t_array"], critical_point_data["10_11"]["PE"], label = "10_11")
ax[0].plot(critical_point_data["t_array"], critical_point_data["00_10"]["PE"], label = "00_10")
ax[0].plot(critical_point_data["t_array"], critical_point_data["01_11"]["PE"], label = "01_11")
ax[0].legend()

ax[1].plot(critical_point_data["t_array"], critical_point_data["00"]["PE"], label = "00")
ax[1].plot(critical_point_data["t_array"], critical_point_data["01"]["PE"], label = "01")
ax[1].plot(critical_point_data["t_array"], critical_point_data["10"]["PE"], label = "10")
ax[1].plot(critical_point_data["t_array"], critical_point_data["11"]["PE"], label = "11")
ax[1].legend()

In [ ]:
plt.plot(critical_point_data["t_array"], critical_point_data["00_01"]["PE"] - critical_point_data["00"]["PE"], label = "bh_faced_by_00")
plt.plot(critical_point_data["t_array"], critical_point_data["00_01"]["PE"] - critical_point_data["01"]["PE"], label = "bh_faced_by_01")
plt.plot(critical_point_data["t_array"], critical_point_data["10_11"]["PE"] - critical_point_data["10"]["PE"], label = "bh_faced_by_10")
plt.plot(critical_point_data["t_array"], critical_point_data["10_11"]["PE"] - critical_point_data["11"]["PE"], label = "bh_faced_by_11")
plt.hlines(y= 0, xmin = 0, xmax=100, linestyles="--")
# plt.plot(critical_point_data["t_array"], critical_point_data["10_11"]["PE"], label = "10_11")
# plt.plot(critical_point_data["t_array"], critical_point_data["00_10"]["PE"], label = "00_10")
# plt.plot(critical_point_data["t_array"], critical_point_data["01_11"]["PE"], label = "01_11")
plt.legend()

### parameter graph

In [ ]:
cfqr.showPamameterTimeGraph(protocol_time_array, protocol_all_time_array)

### potential shape check

In [ ]:
protocol_list_shape_check = [
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_off  , "phi_2_x": phi_2_x_off, "M_12": M_12_on, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name":"M_12_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_off  , "phi_2_x": phi_2_x_off, "M_12": -M_12_on, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name":"M_12_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_on_12  , "phi_2_x": phi_2_x_off, "M_12": -M_12_off, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name": "phi_1x_on_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": -phi_1_x_on_12  , "phi_2_x": phi_2_x_off, "M_12": -M_12_off, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name": "phi_1x_on_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_off  , "phi_2_x": phi_2_x_on_12, "M_12": -M_12_off, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name": "phi_1x_on_only"
    }),
    create_CE_Protocol(10, {
        "phi_1_x": phi_1_x_off  , "phi_2_x": -phi_2_x_on_12, "M_12": -M_12_off, \
        "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "name": "phi_1x_on_only"
    })
]


In [ ]:
_t = 60

critical_points = np.array(cfqr.get_critical_points_at_t_for_a_set_of_parameter(_t, showPotential=False, verbose = False, guess=[(0, 0), (-2, 0), (2, 0), (0,2), (0, -2), (-2, -2), (-2, 2), (2, -2), (2, 2)]))




l = cfqr.plotGraph(t = _t, cutlineValue=critical_points[1][0], x_range=[-3, 3],y_range = [1000, 3000], contour_range = [1000, 2800], showGraph = True)


contour_plt = l["ax0"]
cutline_plt = l["ax1"]
contour_plt.set_aspect('equal')
contour_plt.set_title(f"phi_2x_12 = {-phi_2_x_on_12}")
cutline_plt.set_xlim(-5, 5)
cutline_plt.set_ylim(1000, 3000)

# contour_plt.scatter(critical_points[:, 0], critical_points[:, 1])

# cfqr.plotGraph(t = 50, cutlineValue=-1.96, showGraph=True)
plt.show()

### bifurcation check

In [ ]:
_stepped_time = protocol_all_time_array[::100]
minimum_points_of_00 = [cfqr.get_minimum_at_t_for_a_set_of_parameter(_t, initial_parameter_dict, guess = [(-2, -2)])[0] for _t in _stepped_time]
fig, ax = plt.subplots(1, 2, figsize = [8, 4])

ax[0].scatter(_stepped_time, np.array(minimum_points_of_00)[:, 0])
ax[0].set_xlabel(r't $(\tau_c)$', fontsize = 12)
ax[0].set_ylabel(r'$\varphi_1$', fontsize = 12)

ax[1].scatter(_stepped_time, np.array(minimum_points_of_00)[:, 1])
ax[1].set_xlabel(r't $(\tau_c)$', fontsize = 12)
ax[1].set_ylabel(r'$\varphi_2$', fontsize = 12)
fig.tight_layout()